In [ ]:
import pandas as pd
from openai import OpenAI
import tiktoken
import time
import os
from openai import OpenAI
client = OpenAI(
    api_key=os.getenv(""),
    base_url="",
)
# 最大 token 限制
MAX_TOKENS = 8000  # 将 token 限制改为 4000

# 使用 tiktoken 进行 token 数量估算
def count_tokens(text):
    """估算文本的 token 数"""
    enc = tiktoken.get_encoding("cl100k_base")  # 获取与 GPT-3.5 和 GPT-4 兼容的 tokenizer
    tokens = enc.encode(text)
    return len(tokens)

# 发送请求并处理 token 限制的 prompt 函数
def prompt(s, history,llm_name="Baichuan4-Air"):
    """发送请求，确保 token 数在模型限制范围内"""
    # 将 s0 作为系统角色消息添加到历史记录中（如果尚未添加）
    if not history[0]["content"]:
        history[0]["content"] = s0
    
    # 将用户输入添加到历史记录中
    context = history + [{"role": "user", "content": s}]
    
    # 计算当前 context 的 token 总数
    total_tokens = sum(count_tokens(item['content']) for item in context)
    print(f"当前总 token 数：{total_tokens}")

    # 如果 token 数量超过限制，裁剪历史记录
    while total_tokens > MAX_TOKENS:
        context.pop(1)  # 移除最早的非系统消息（s0 已保留）
        total_tokens = sum(count_tokens(item['content']) for item in context)
        print(f"裁剪后总 token 数：{total_tokens}")

    try:
        # 调用千帆接口
        response = client.chat.completions.create(
            model=llm_name,  # 使用选择的模型
            messages=context  # 传递消息
        )
        
        # 获取大模型的反馈内容
        feedback = response.choices[0].message.content.strip('"')
        print(feedback)
        
        return context + [{"role": "assistant", "content": feedback}]
    
    except Exception as e:
        print(f"Error: {e}")
        print("等待 60 秒后重试...")
        time.sleep(60)  # 等待后重试
        return prompt(s, history,llm_name)  # 重试请求



# 初始化第一个提示信息
s0="""
**角色：作为一个图书领域的AI助手，你非常擅长从图书直播文本中提取关键信息，并且擅长从直播文本中提取到段落文本介绍的图书信息。
**任务：我将提供给你真实的直播带货文本，请你理解主播的语意意图，从文本中识别出这个段落正在介绍哪本图书。如果你无法识别，请直接输出“废话”。
**约束：1. 请你保证输出结果的书名是正确的，没有错别字的，现实中真实存在的图书名称；
       2. 请你直接输出书名，不要带书名号。
请你直接在label中给出最终识别的图书名称，不要做解释：
"""

# 调用 prompt 函数，并确保 s0 只输入一次
history = [{"role": "system", "content": ""}]# 初始为空，稍后填写 s0



# 加载数据
input_file = "/product_name.xlsx"
output_file = "/model_name.xlsx"

df = pd.read_excel(input_file)
sentences = df['句子']
results = []

template = """
text:{}
label:
"""

batch_size = 10  # 每批处理200个句子
delay_between_batches = 1  # 每批次处理间隔10秒

# 遍历sentences并调用prompt函数
for i in range(0, len(sentences), batch_size):
    batch_sentences = sentences[i:i+batch_size]
    
    for sentence in batch_sentences:
        s = template.format(sentence)  # 格式化每个句子
        
        # 调用 prompt 函数
        history = prompt(s, history)
        result = history[-1]['content']
        
        # 添加结果到 results 列表
        results.append({'original': sentence, 'result': result})
        
        # 每个请求间的延迟
        time.sleep(0.1)
    
    # 每批次处理后，等待一段时间
    print(f"批次 {i // batch_size + 1} 处理完成，暂停 {delay_between_batches} 秒...")
    time.sleep(delay_between_batches)

# 将结果列表转换为 DataFrame
results_df = pd.DataFrame(results)

# 保存结果到新的xlsx文件
results_df.to_excel(output_file, index=False)

print(f"处理完成，结果已保存至 {output_file}")

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 读取文件
truth_df = pd.read_excel("/product_name_tage.xlsx")
pred_df = pd.read_excel("/model_name.xlsx")

# 确保句子列名称一致，并且按句子对齐
truth_df = truth_df[['句子', 'result']]  # 选择需要的列
pred_df = pred_df[['句子', 'result']]  # 选择需要的列

# 合并两个数据集，按句子列对齐
df_merged = pd.merge(truth_df, pred_df, on='句子', suffixes=('_true', '_pred'))

# 提取真实结果和预测结果列
y_true = df_merged['result_true']
y_pred = df_merged['result_pred']

# 计算评估指标
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)  # 设置zero_division=0避免警告
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

# 输出评估指标
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# 如果需要更详细的分类报告，可以使用 classification_report
report = classification_report(y_true, y_pred, zero_division=0)
print("Classification Report:\n", report)